In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import yfinance as yf


ModuleNotFoundError: No module named 'tensorflow.python'

In [2]:
# Download historical stock price data (e.g., Apple Inc.)
df = yf.download('AAPL', start='2015-01-01', end='2024-01-01')
df = df[['Close']]  # We are focusing on the 'Close' price
df.head()


NameError: name 'yf' is not defined

In [4]:
# Convert the data to a numpy array
data = df.values

# Normalize the data to be within the range of 0 to 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Define the training data size (80% of the data)
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]

# Prepare the data for LSTM model
def create_dataset(dataset, look_back=60):
    X, Y = [], []
    for i in range(look_back, len(dataset)):
        X.append(dataset[i-look_back:i, 0])
        Y.append(dataset[i, 0])
    return np.array(X), np.array(Y)

look_back = 60  # Number of previous days to consider for prediction
X_train, y_train = create_dataset(train_data, look_back)

# Reshape the data to be 3D for LSTM [samples, time steps, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)


NameError: name 'df' is not defined

In [ ]:
# Initialize the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=50)


In [ ]:
# Create the testing data set
test_data = scaled_data[train_size - look_back:]

# Create the test dataset
X_test, y_test = create_dataset(test_data, look_back)

# Reshape the data to be 3D for LSTM
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Predict the stock prices
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)  # Undo scaling

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(data[train_size + look_back:], predictions))
print(f"Root Mean Squared Error: {rmse}")


In [ ]:
# Plot the data
train = df[:train_size]
valid = df[train_size:]
valid['Predictions'] = predictions

plt.figure(figsize=(16,8))
plt.title('Stock Price Prediction using LSTM')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Actual', 'Predictions'], loc='lower right')
plt.show()


In [ ]:
# Simple trading strategy based on predictions
valid['Buy Signal'] = valid['Predictions'] < valid['Close']
valid['Sell Signal'] = valid['Predictions'] > valid['Close']

# Visualizing buy/sell signals
plt.figure(figsize=(16,8))
plt.plot(valid['Close'], label='Close Price')
plt.plot(valid['Predictions'], label='Predictions')
plt.plot(valid.loc[valid['Buy Signal']].index, valid['Close'][valid['Buy Signal']], '^', markersize=10, color='g', label='Buy Signal')
plt.plot(valid.loc[valid['Sell Signal']].index, valid['Close'][valid['Sell Signal']], 'v', markersize=10, color='r', label='Sell Signal')
plt.legend(loc='lower right')
plt.title('Stock Price with Buy/Sell Signals')
plt.show()
